In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
#from esda.moran import Moran
from libpysal.weights.contiguity import Queen
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [22]:
#path1 = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\Datasets\som_adm_ocha_itos_20230308_shp\som_admbnda_adm1_ocha_20230308.shp"
path2 = r"/home/sara/Documenti/GitHub/Climate-and-conflict/Datasets/som_adm_ocha_itos_20230308_shp/som_admbnda_adm1_ocha_20230308.shp"
states_gdf = gpd.read_file(path2)

#file1 = r"C:\Users\PcLaptop\Dropbox (Politecnico Di Torino Studenti)\Untitled Folder\distance_df.csv"
file2 = r"/home/sara/Documenti/GitHub/Climate-and-conflict/Datasets/ACLED_1997-01-01-2023-07-18_Somalia.csv"
dist = pd.read_csv(file2)

#csv1 = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\df_with_dummies.csv"
csv2 = r"/home/sara/Documenti/GitHub/Climate-and-conflict/df_CRU4.csv"
df_with_dummies = pd.read_csv(csv2)

In [17]:
# add column with gaussian noise in df_with_dummies
#df_with_dummies['noise'] = np.random.normal(0, 10, df_with_dummies.shape[0])

In [18]:
y_var_name = 'conflicts'
X_var_names = ['TA','PA','DL']

In [19]:
#from month number to full month name
month_dict = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6:'June', 7:'July', 8:'August', 9:'September', 10:'October', 11:'November', 12:'December'}
df_with_dummies['month'] = df_with_dummies['month'].map(month_dict)

In [20]:
# Regression expression for OLS with dummies

unit_names = df_with_dummies['admin1'].unique().tolist()
unit_names.sort()
unit_names_t = df_with_dummies['month'].unique().tolist()
unit_names_mr = (df_with_dummies['admin1'] + df_with_dummies['month']).unique().tolist()

lsdv_expr = y_var_name + ' ~ '
i = 0
for X_var_name in X_var_names:
    if i > 0:
        lsdv_expr = lsdv_expr + ' + ' + X_var_name
    else:
        lsdv_expr = lsdv_expr + X_var_name
    i = i + 1
for dummy_name in unit_names[:-1]:
   lsdv_expr = lsdv_expr + ' + ' + dummy_name
for dummy_name_t in unit_names_t[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name_t
#for dummy_name_mr in unit_names_mr[:-1]:
   # lsdv_expr = lsdv_expr + ' + ' + dummy_name_mr
#lsdv_expr=lsdv_expr + ' - 1' 
print('Regression expression for OLS with dummies=' + lsdv_expr)

Regression expression for OLS with dummies=conflicts ~ TA + PA + DL + Awdal + Bakool + Banadir + Bari + Bay + Galgaduud + Gedo + Hiraan + Lower_Juba + Lower_Shabelle + Middle_Juba + Middle_Shabelle + Mudug + Nugaal + Sanaag + Sool + Togdheer + January + February + March + April + May + June + July + August + September + October + November


In [21]:
lsdv_model = smf.ols(formula=lsdv_expr, data=df_with_dummies)
lsdv_model_results = lsdv_model.fit()
print(lsdv_model_results.summary())

                            OLS Regression Results                            
Dep. Variable:              conflicts   R-squared:                       0.429
Model:                            OLS   Adj. R-squared:                  0.426
Method:                 Least Squares   F-statistic:                     135.4
Date:                Fri, 15 Sep 2023   Prob (F-statistic):               0.00
Time:                        17:10:23   Log-Likelihood:                -21037.
No. Observations:                5616   AIC:                         4.214e+04
Df Residuals:                    5584   BIC:                         4.235e+04
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.0571      0.751     